# Get club elos
Let's get a elos of clubs using standard elo method.
Club elos.com use different name from ours, so although it will be more accurate in terms of an actual elo, it will be better to use ours cuz of naming issues?

In [1]:
import os
from pathlib import Path
import pandas as pd
import datetime
from datetime import datetime

In [42]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

# # Example of using the path to the data directory
# csv_file = DATA_DIR / 'your_file.csv'
# print(csv_file)

# import all files in Data folder and read into dataframes
dataframes=[]


# Below is just to suppress warnings...
competitions_df = pd.DataFrame
appearances_df = pd.DataFrame
player_valuations_df = pd.DataFrame
game_events_df = pd.DataFrame
players_df= pd.DataFrame
games_df = pd.DataFrame
club_games_df = pd.DataFrame
clubs_df = pd.DataFrame

# Actual reading csv flies
for dirpath, dirname, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirpath,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported')



game_lineups_df (2191911, 10)
competitions_df (44, 11)
appearances_df (1610495, 13)
player_valuations_df (486009, 5)
game_events_df (761513, 10)
transfers_df (77585, 10)
players_df (32392, 23)
games_df (69635, 23)
club_games_df (139270, 11)
club_elos_df (6386, 9)
clubs_df (439, 17)
Data imported


## 2. Clean up games df so that it only cotains game between clubs that exists in our clubs_df

In [3]:
clubs_df.dtypes

club_id                      int64
club_code                   object
name                        object
domestic_competition_id     object
total_market_value         float64
squad_size                   int64
average_age                float64
foreigners_number            int64
foreigners_percentage      float64
national_team_players        int64
stadium_name                object
stadium_seats                int64
net_transfer_record         object
coach_name                 float64
last_season                  int64
filename                    object
url                         object
dtype: object

In [6]:
home_exists = games_df['home_club_id'].isin(clubs_df['club_id'])
away_exists = games_df['away_club_id'].isin(clubs_df['club_id'])
filtered_games_df = games_df[home_exists & away_exists]
filtered_games_df.shape

(55256, 23)

In [43]:
# Back up games df
games_df_backup = games_df.copy()


In [47]:
club_games_merged_df = pd.merge(filtered_games_df, club_games_df, on='game_id', how='inner')

Now, let's clean 
- Change data type to valid one
- Sort the games by time order
- Loop through each game and update elo of clubs 

In [48]:
# games_df = filtered_games_df.copy()

# Change data type to datetime format
club_games_merged_df['date'] = club_games_merged_df['date'].str.strip()
club_games_merged_df['date'] = pd.to_datetime(club_games_merged_df['date'], errors='coerce', format='%Y-%m-%d')
club_games_merged_df.dropna(subset=['date'], inplace=True)

# Sort by date
club_games_merged_df = club_games_merged_df.sort_values(by=['date'])


In [49]:
# Init. club elo to certain value 
clubs_df['elo'] = 1000
test_games_df = club_games_merged_df.head(1000)

Starting from first game lets use standard elo to modify elos of team

In [50]:
import numpy as np

In [51]:
def expected_chance_of_success(home_team_elo, away_team_elo) -> int:
    """Calculate chance of success E_home

    Args:
        home_team_elo (int): _description_
        away_team_elo (int): _description_

    Returns:
        int : Expected chance of home team winning
    """
    return 1 / (1 + 10 ** ((away_team_elo - home_team_elo) / 400))

def calculate_new_elo(home_team_elo, away_team_elo, result, K=30):
    """Calculate new elo 

    Args:
        home_team_elo (_type_): _description_
        away_team_elo (_type_): _description_
        result (_type_): result (should be 1, 0, -1)
        K (int, optional): _description_. Defaults to 30.

    Returns:
        tuple (int, int): new elos of home and away team 
    """
    if result not in [-1, 0, 1]:
        raise AssertionError('Invalid result value passed!')
    
    
    expected_home_team = expected_chance_of_success(home_team_elo, away_team_elo)
    expected_away_team = 1 - expected_home_team
    
    new_home_elo = home_team_elo + K * (result - expected_home_team)
    new_away_elo = away_team_elo + K * ((1 - result) - expected_away_team)
    
    return new_home_elo, new_away_elo
    


In [76]:
for idx, row in test_games_df.iterrows():
    home_club_id = row['home_club_id']
    away_club_id = row['away_club_id']

    # Get current ELOs for home and away clubs from clubs_df
    home_elo = clubs_df.loc[clubs_df['club_id'] == home_club_id, 'elo'].values[0]
    away_elo = clubs_df.loc[clubs_df['club_id'] == away_club_id, 'elo'].values[0]

    # Get result for the game (assuming 1 for home win, 0 for away win, 0.5 for draw)
    game_result = row['is_win']  # Adjust this based on your game logic

    # Update ELOs based on the result
    new_home_elo, new_away_elo = calculate_new_elo(home_elo, away_elo, game_result)

    clubs_df.loc[clubs_df['club_id'] == home_club_id, 'elo'] = new_home_elo
    clubs_df.loc[clubs_df['club_id'] == away_club_id, 'elo'] = new_away_elo

/var/folders/nh/kbd4vyw93l9_cfg59h74rpwh0000gn/T/ipykernel_1657/4219161292.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1001.2919947606664' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clubs_df.loc[clubs_df['club_id'] == home_club_id, 'elo'] = new_home_elo


In [77]:
clubs_df

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,elo
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...,1000.000000
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...,1000.000000
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...,994.359621
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...,971.398345
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...,943.485454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,724,fc-volendam,FC Volendam,NL1,NaN,25,23.4,6,24.0,2,Kras Stadion,7384,+€800k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-volendam/st...,1000.000000
435,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,26,26.8,15,57.7,19,Gewiss Stadium,21747,+€18.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...,1001.291995
436,85465,karpaty-lviv-2020-,FK Karpaty Lviv,UKR1,NaN,28,24.8,6,21.4,1,Ukraina,28051,€-1.48m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/karpaty-lviv-2...,1000.000000
437,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,24,25.8,16,66.7,3,Estádio C. J. de Almeida Freitas,6150,+€2.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...,1022.755014


In [13]:
test_games_df.columns

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type'],
      dtype='object')

## Using standard ELO


In [3]:
clubs_df['elo'] = 1000

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,elo
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...,1000
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...,1000
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...,1000
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...,1000
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,724,fc-volendam,FC Volendam,NL1,NaN,25,23.4,6,24.0,2,Kras Stadion,7384,+€800k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-volendam/st...,1000
435,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,26,26.8,15,57.7,19,Gewiss Stadium,21747,+€18.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...,1000
436,85465,karpaty-lviv-2020-,FK Karpaty Lviv,UKR1,NaN,28,24.8,6,21.4,1,Ukraina,28051,€-1.48m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/karpaty-lviv-2...,1000
437,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,24,25.8,16,66.7,3,Estádio C. J. de Almeida Freitas,6150,+€2.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...,1000


In [5]:
club_games_df.head()
games_df.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league
3,2321152,L1,2013,11. Matchday,2013-11-02,44,33,0,2,7.0,...,Olympiastadion Berlin,69277.0,Günter Perl,https://www.transfermarkt.co.uk/hertha-bsc_fc-...,4-2-3-1,4-2-3-1,Hertha BSC,FC Schalke 04,0:2,domestic_league
4,2321205,L1,2013,17. Matchday,2013-12-21,41,39,2,3,14.0,...,Volksparkstadion,50000.0,Bastian Dankert,https://www.transfermarkt.co.uk/hamburger-sv_1...,4-2-3-1,4-4-2 Diamond,Hamburger SV,1. Fußball- und Sportverein Mainz 05,2:3,domestic_league


Use games_df to analyse through game from oldest to newest

In [6]:
# Change data type to datetime format
games_df['date'] = games_df['date'].str.strip()
games_df['date'] = pd.to_datetime(games_df['date'], errors='coerce', format='%Y-%m-%d')
games_df.dropna(subset=['date'], inplace=True)

# Sort by date
games_df = games_df.sort_values(by=['date'])
games_df.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
14610,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,Gundadalur,650.0,Andris Treimanis,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:6,international_cup
14605,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,Windsor Park,1341.0,Wim Smet,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
43075,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,Stade Jos Nosbaum,1070.0,Alan Mario Sant,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,7:0,international_cup
10890,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,Milano Arena,2000.0,Eitan Shmuelevic,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
46883,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,Tony Bezzina Stadium,1136.0,Johnny Casanova,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup


In [13]:
games_df_backup = games_df.copy()
games_df = games_df.copy().head(20)
games_df


,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
14610,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,Gundadalur,650.0,Andris Treimanis,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:6,international_cup
14605,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,Windsor Park,1341.0,Wim Smet,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
43075,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,Stade Jos Nosbaum,1070.0,Alan Mario Sant,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,7:0,international_cup
10890,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,Milano Arena,2000.0,Eitan Shmuelevic,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
46883,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,Tony Bezzina Stadium,1136.0,Johnny Casanova,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup
43081,2235780,ELQ,2012,First Round 1st leg,2012-07-05,1189,8036,1,0,NaN,...,Richmond Park,1652.0,Igor Satchi,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,1:0,international_cup
14612,2235772,ELQ,2012,First Round 1st leg,2012-07-05,10262,18506,2,2,NaN,...,Tony Bezzina Stadium,410.0,Ivaylo Stoyanov,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,2:2,international_cup
14608,2233850,ELQ,2012,First Round 1st leg,2012-07-05,10854,327,0,1,NaN,...,Estadi Comunal,500.0,Sergejus Slyva,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:1,international_cup
50662,2235539,ELQ,2012,First Round 1st leg,2012-07-05,282,10604,5,2,NaN,...,Asim Ferhatović - Hase,8000.0,Goran Spirkoski,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,5:2,international_cup
60209,2235542,ELQ,2012,First Round 1st leg,2012-07-05,1101,10603,8,0,NaN,...,Borås Arena,2653.0,John Beaton,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup


In [21]:
games_df_backup

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
14610,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,Gundadalur,650.0,Andris Treimanis,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:6,international_cup
14605,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,Windsor Park,1341.0,Wim Smet,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
43075,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,Stade Jos Nosbaum,1070.0,Alan Mario Sant,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,7:0,international_cup
10890,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,Milano Arena,2000.0,Eitan Shmuelevic,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
46883,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,Tony Bezzina Stadium,1136.0,Johnny Casanova,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup
43081,2235780,ELQ,2012,First Round 1st leg,2012-07-05,1189,8036,1,0,NaN,...,Richmond Park,1652.0,Igor Satchi,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,1:0,international_cup
14612,2235772,ELQ,2012,First Round 1st leg,2012-07-05,10262,18506,2,2,NaN,...,Tony Bezzina Stadium,410.0,Ivaylo Stoyanov,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,2:2,international_cup
14608,2233850,ELQ,2012,First Round 1st leg,2012-07-05,10854,327,0,1,NaN,...,Estadi Comunal,500.0,Sergejus Slyva,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:1,international_cup
50662,2235539,ELQ,2012,First Round 1st leg,2012-07-05,282,10604,5,2,NaN,...,Asim Ferhatović - Hase,8000.0,Goran Spirkoski,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,5:2,international_cup
60209,2235542,ELQ,2012,First Round 1st leg,2012-07-05,1101,10603,8,0,NaN,...,Borås Arena,2653.0,John Beaton,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup


In [ ]:
def find_club(club_id: int, games_df: pd.DataFrame):
    pass


In [19]:
# Merge for home_club_id
home_merge = pd.merge(games_df, clubs_df, left_on='home_club_id', right_on="club_id", how='left', suffixes=('', '_home'))
home_merge = home_merge.drop('club_id', axis=1).rename(columns={'club_name': 'home_club_name'})

home_merge.head()



,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url_home,elo
0,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:

# Merge for away_club_id
final_merge = pd.merge(home_merge, clubs_df, left_on='away_club_id', right_on='club_id', how='left', suffixes=('', '_away'))
final_merge = final_merge.drop('club_id', axis=1).rename(columns={'club_name': 'away_club_name'})
final_merge.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,coach_name_away,last_season_away,filename_away,url_away,elo_away
0,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
